# FOLLOWER RECOMMENDER

IMPORT NECESSARY LIBRARIES

In [26]:
import numpy as np
import pandas as pd
import os

In [27]:
user = pd.read_csv("data/users.csv")
user.columns

Index(['id', 'name', 'username', 'email', 'image', 'provider', 'provider_id',
       'password', 'remember_token', 'created_at', 'updated_at', 'short_bio'],
      dtype='object')

In [28]:
user.drop(labels=['image', 'provider', 'provider', 'provider_id', 'password', 'remember_token', 'created_at', 'updated_at'], axis=1, inplace=True )
user.head(10)

,id,name,username,email,short_bio
0,1,Eniayomi Oluwaseyi,eniayomi,oluwaseyieniayomi@gmail.com,Software Developer | DevOPs Engineer
1,2,Elijah Okokon,DMatrix,okoelijah@gmail.com,Web Developer
2,3,Jeffrey Ogah,jeff.ogah,jeff.ogah@gmail.com,Front End Developer | React Developer | Mentor...
3,4,Oluwaseyi Oluwapelumi,nathan,nathanoluwaseyi@gmail.com,| Software Developer | DevOps Engineer | @linu...
4,5,PoRH,lamar,paulchibiukeigweze@gmail.com,I Am lamar and you don't think am real?
5,6,Seyi Onifade,xyluz,hngtechapps@gmail.com,My Open Diary
6,7,Mark Essien,markessien,markessien@gmail.com,An engineer
7,8,Alex Moses,ahlesswywk,ahlesswywk@gmail.com,"Software Developer hotelsng, Ex. Frontend deve..."
8,9,Edmund Ekott,26th_edmund,edmund.timfon@gmail.com,Front-end Engineer 💻 | FIFA advocate 🎮 | Ninja 🥋
9,10,Stefan aGz,stefanpongrz,stefanpongrz@gmail.com,Here


In [29]:
user.shape

(2293, 5)

In [30]:
user['short_bio'].head(5)

0                 Software Developer | DevOPs Engineer
1                                        Web Developer
2    Front End Developer | React Developer | Mentor...
3    | Software Developer | DevOps Engineer | @linu...
4              I Am lamar and you don't think am real?
Name: short_bio, dtype: object

In [31]:
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer

In [32]:
#Define a TF-IDF Vectorizer Object. Remove all english stop words such as 'the', 'a'
tfidf = TfidfVectorizer(stop_words='english')

In [33]:
#Replace NaN with an empty string
user['short_bio'] = user['short_bio'].fillna('')

In [34]:
#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(user['short_bio'])

In [35]:
#Output the shape of tfidf_matrix
# this produces the number of different words used by user in the short_bio
tfidf_matrix.shape

(2293, 2242)

In [36]:
# Import linear_kernel
from sklearn.metrics.pairwise import linear_kernel

In [37]:
# Compute the cosine similarity matrix
consine_sim = linear_kernel(tfidf_matrix, tfidf_matrix, True)

In [38]:
#Construct a reverse map of indices and user name
indices = pd.Series(user.index, index=user['id'])

In [39]:
def create_new_db(x):
    """
    this function takes the result from get_followers and transforms to a DB with other info
    paramx: recommendation Series
    """
    y = x.copy()
    z = x.copy()
    y.update(user['name'])
    z.update(user['short_bio'])
    xdf=pd.DataFrame(x)
    ydf=pd.DataFrame(y)
    zdf=pd.DataFrame(z)
    ydf.rename(columns={'id':'Name'}, inplace=True)
    xdf.rename(columns={'id':'User_Id'}, inplace=True)
    zdf.rename(columns={'id':'short_bio'}, inplace=True)
    frames = [xdf,ydf,zdf]
    new_df = pd.concat(frames, axis=1)
    new_df.set_index('User_Id', inplace=True)
    return new_df

In [40]:
# Function that takes in user name as input and outputs most similar users
def get_followers(id, consine_sim=consine_sim):
    # Get the index of the users that matches the user
    idx = indices[id]
    # Get the pairwsie similarity scores of all users with that user
    sim_scores = list(enumerate(consine_sim[idx]))
    # Sort the users based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Get the scores of the 10 most similar users
    sim_scores = sim_scores[1:11]
     # Get the user indices
    follow_indices = [i[0] for i in sim_scores]
      # Return the top 10 most similar user
    follower_id = user['id'].iloc[follow_indices]
    # Create a new data frame containing the user id and name 
    follower_rec = create_new_db(follower_id)
    return follower_rec

In [59]:
# Use this function to get details of the user with their user id
def check_user(id):
    name = user.loc[user['id'] == id, 'name']
    bio = user.loc[user['id'] == id, 'short_bio']
    return name.iloc[0], bio.iloc[0]

In [60]:
check_user(23)

('Oluwatomisin Lalude', 'Web Developer')

In [61]:
# Run Get_followers with user id as input
get_followers(23)

,Name,short_bio
User_Id,,
23,Oluwatomisin Lalude,Web Developer
66,Kev Chike,Web Developer
78,Emmanuel Ezenwigbo,Web developer
338,Stephen Oluwafemi,Web Developer
362,Hafizah Muhyideen,I am a web developer
725,Tochukwu Nwosu,I am a Web Developer
857,benjamin chibuzor,Web Developer
1012,OMIDIORA EMMANUEL,Web Developer
1167,Ogunjuyigbe Oluwaseyi,Web developer


In [62]:
user.loc[user['id'] == 23, 'short_bio'].item()


'Web Developer'